In [ ]:
# hidden-but-run
import sys
sys.path.insert(0, "..")

# Plotting maps

Here are examples to plot geographic data using [plotly](https://plotly.com) and [matplotlib](https://matplotlib.org/). Matplotlib is probably the choice if you need a rendered image. 
Plotly creates interactive plots and has a *more modern* interface.  

To handle the different geo-types returned by elasticsearch we first look at conversion utilities. [Skip it](#geo-centroid) if you just want to see pretty images.

## Coordinate conversion

A metric aggregation like :link:`geo_centroid <Aggregation.metric_geo_centroid>` already returns 
[latitude and longitude](https://en.wikipedia.org/wiki/Geographic_coordinate_system#Latitude_and_longitude)
values.

Bucket-aggregations like :link:`geotile_grid <Aggregation.agg_geotile_grid>` 
and :link:`geohash_grid <Aggregation.agg_geohash_grid>` return keys that can be **mapped**
to geo-coordinates.

### map-tiles

The :link:`geotile_grid <Aggregation.agg_geotile_grid>` aggregation uses *map-tiles* 
([wikipedia](https://en.wikipedia.org/wiki/Tiled_web_map)) as bucket keys. 
They represent `zoom`/`x`/`y` as seen below: 

In [ ]:
from elastipy import Search

s = Search(index="elastipy-example-car-accidents")

agg = s.agg_geotile_grid("tiles", field="location", precision=6)

agg.execute().to_dict()

To convert the keys to [geo-coordinates](https://en.wikipedia.org/wiki/Geographic_coordinate_system#Latitude_and_longitude) we can use a helper function in elastipy:

In [ ]:
from elastipy import geotile_to_lat_lon

{
    geotile_to_lat_lon(key): value
    for key, value in agg.items()
}

Becaue the tiles are actually **areas** the latitude and longitude just represent a single point within the area. The point can be defined as the ``offset`` parameter and defaults to ``(.5, .5)`` which is the center of the tile.

Here we print the *top-left* and *bottom-right* coordinates for each map-tile:

In [ ]:
for key, value in agg.items():
    tl = geotile_to_lat_lon(key, offset=(0, 1))
    bl = geotile_to_lat_lon(key, offset=(1, 0))
    print(f"{tl} - {bl}: {value}")


### geohash

The :link:`geohash_grid <Aggregation.agg_geohash_grid>` aggregation returns *geohash* 
([wikipedia](https://en.wikipedia.org/wiki/Geohash)) bucket keys.

In [ ]:
from elastipy import Search

s = Search(index="elastipy-example-car-accidents")

agg = s.agg_geohash_grid("tiles", field="location", precision=2)

agg.execute().to_dict()

The [pygeohash](https://github.com/wdm0006/pygeohash) package can be used to translate them:

In [ ]:
import pygeohash

{
    pygeohash.decode(key): value
    for key, value in agg.items()
}

For convenience the pygeohash function is wrapped by `elastipy.geohash_to_lat_lon`.

## plotly backend

The [plotly python library](https://plotly.com/python/) enables creating browser-based plots in python. It supports a range of [map plots](https://plotly.com/python/maps/). In particular the [mapbox](https://www.mapbox.com/) based plots are interesting because they use WebGL and render quite fast even for a large number of items. 

### geo-centroid

Let's plot an overview of the german car accidents (included in elastipy [examples](https://github.com/netzkolchose/elastipy/blob/development/examples/accidents_export.py)). 

In [ ]:
s = Search(index="elastipy-example-car-accidents")
agg = s.agg_terms("city", field="city", size=10000)
agg = agg.metric_geo_centroid("location", field="location")

df = agg.execute().df()
print(f"{df.shape[0]} cities")
df.head()

The :link:`geo_centroid <Aggregation.metric_geo_centroid>` aggregation above returns the center coordinate of all accidents within a city. (It's not necessarily the center of the city but the [centroid](https://en.wikipedia.org/wiki/Centroid) of all accidents that are assigned to the city.)

Below we pass the pandas DataFrame to the plotly express function and tell it the names of the latitude and longitude columns. The number of accidents per city is also used for the color and size of the points.

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(
    df, 
    lat="location.lat", lon="location.lon", 
    color="city.doc_count", opacity=.5, size="city.doc_count",
    zoom=4.8,
    mapbox_style="carto-positron",
    hover_data=["city"],
    labels={"city.doc_count": "number of accidents"},
    
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

The most amazing thing we should notice is that the federal state Mecklenburg-Vorpommern does not have any accidents! 🍀

### density heatmap

The plotly express tools are just lovely ♥ ❤️ ♥ ❤️ 

In [ ]:
fig = px.density_mapbox(
    df, 
    lat="location.lat", lon="location.lon", 
    z="city.doc_count", 
    zoom=4.8,
    mapbox_style="carto-positron",
    hover_data=["city"],
    labels={"city.doc_count": "number of accidents"},
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

### geohash_grid aggregation

Below is the same data-set but aggregated with the :link:`geohash_grid <Aggregation.agg_geohash_grid>` aggregation. 

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

from elastipy import geotile_to_lat_lon 

s = Search(index="elastipy-example-car-accidents")
agg = s.agg_geotile_grid("location", field="location", precision=10, size=1000)

df = agg.execute().df()

# put lat and lon columns into dataframe
df[["lat", "lon"]] = list(df["location"].map(geotile_to_lat_lon))
print(df.head())

fig = px.scatter_mapbox(
    df, 
    lat="lat", lon="lon", 
    color="location.doc_count", opacity=.5, size="location.doc_count",
    mapbox_style="carto-positron",
    zoom=5,
    labels={"location.doc_count": "number of accidents"},
    
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

### geotile_grid aggregation

Let's see if we can do something with the :link:`geotile_grid aggregation <Aggregation.agg_geotile_grid>`. The lengthy function in the middle builds a list of lines connecting each corner in each returned map-tile.

Unfortunately, the ``fillcolor`` in mapbox can only be one fixed color and does not support color scaling (like the [marker](https://plotly.com/python/reference/scattermapbox/#scattermapbox-marker-colorscale)).   
If you know differently or have an idea how to color the rendered tiles according to aggregated values, 
[please let me know](https://github.com/netzkolchose/elastipy/issues).

In [ ]:
import plotly.graph_objects as go
import plotly.colors

from elastipy import Search, geotile_to_lat_lon 

s = Search(index="elastipy-example-car-accidents")

agg = s.agg_geotile_grid(
    "location", 
    field="location", precision=8, size=1000,
)
agg.execute()

lat, lon = [], []
for key, value in agg.items():
    tl = geotile_to_lat_lon(key, offset=(0, 1))
    tr = geotile_to_lat_lon(key, offset=(1, 1))
    bl = geotile_to_lat_lon(key, offset=(0, 0))
    br = geotile_to_lat_lon(key, offset=(1, 0))
    lat += [tl[0], tr[0], br[0], bl[0], tl[0], None]
    lon += [tl[1], tr[1], br[1], bl[1], tl[1], None]

fig = go.Figure(go.Scattermapbox(
    lat=lat, lon=lon,
    fill="toself",
    fillcolor="rgba(0,0,0,.1)",
))
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=5,
        center=dict(lat=51., lon=10.3),
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

## matplotlib backend

Matplotlib does not come with specific geo functionality out-of-the-box. Instead a couple of additional libraries must be used.

### geotile_grid aggregation

Here is an example using [geopandas](https://geopandas.org/index.html). It extends the :link:`pandas.DataFrame` with the [geopandas.GeoDataFrame](https://geopandas.org/data_structures.html#geodataframe) class.

The GeoDataFrame will pick the `"geometry"` column from a DataFrame by default. The values must be [shapely](https://github.com/Toblerity/Shapely) geometries.

In [ ]:
# First we get a shapefile for the german administrative areas
# This cell will not be in the docs because of below tag:
# run-but-hide

import os
import zipfile
from examples.helper import get_web_file

filename_zip = get_web_file(
    "https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_DEU_shp.zip",
    "gadm36_DEU_shp.zip",
)
path = os.path.dirname(filename_zip)
path

with zipfile.ZipFile(filename_zip, "r") as zip_ref:
    zip_ref.extractall(path)


In [ ]:
from shapely.geometry import Point
import geopandas
import matplotlib.pyplot as plt
import matplotlib.colors

from elastipy import Search, geotile_to_lat_lon

s = Search(index="elastipy-example-car-accidents")
agg = s.agg_geotile_grid("location", field="location", precision=10)

df = agg.execute().df()

# take hash from location column, 
#   convert to latitude and longitude
#   and create a shapely.Point 
#   (which expects longitude, latitude)
df["geometry"] = df.pop("location").map(
    lambda v: Point(geotile_to_lat_lon(v)[::-1])
)

# have a color for each point with matplotlib tools
cmap = plt.cm.magma
norm = matplotlib.colors.Normalize(
    df["location.doc_count"].min(), df["location.doc_count"].max()
)
df["color"] = df["location.doc_count"].map(lambda v: cmap(norm(v))[:3] + (.5,))

gdf = geopandas.GeoDataFrame(df)

fig, ax = plt.subplots(figsize=(10, 10))
# plot a shapefile from https://biogeo.ucdavis.edu/data/gadm3.6
geopandas.read_file("cache/gadm36_DEU_1.shp").plot(ax=ax, color="#e0e0e0")

gdf.plot(
    c=gdf["color"], markersize=gdf["location.doc_count"] / 3,
    aspect=1.3, 
    ax=ax,
)